## Cosmic Ray Parameters of interest
<ol>
    <li><a href="#total_energy">Total energy per cosmic ray</a></li>
    <li><a href="#cr_shape_size">Size and shape of cosmic ray</a></li>
    <li><a href="#normal_incidence">Angle of incidence wrt the normal of the ccd chip</a></li>
    <li><a href="#polar_angle">Angle wrt to the longest axis of the ccd chip</a></li>
    <li><a href="#cr_incidence">Cosmic ray event rate (cr/sec)</a></li>
</ol>

In [ ]:
%matplotlib notebook
from astropy.io import fits
from astropy.visualization import LinearStretch, ZScaleInterval, SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from mpl_toolkits.axes_grid1 import make_axes_locatable
import datetime as dt
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
plt.style.use('ggplot')
import numpy as np
import pandas as pd
from photutils.utils import random_cmap
from scipy import ndimage
import sys
import time

# Local modules
from CosmicRayLabel import CosmicRayLabel
from ComputeStats import ComputeStats

In [ ]:
fname = '../data/jd4yacw0q_flt.fits'
with fits.open(fname) as hdu:
    sci2 = hdu[1].data
    dq2 = hdu[3].data
    sci1 = hdu[4].data
    dq1 = hdu[6].data
dq = np.concatenate([dq2, dq1])
sci = np.concatenate([sci2, sci1])
print(dq.shape, sci.shape)

# Initialize a scaling for plots of CR's
norm = ImageNormalize(sci, stretch=LinearStretch(), interval=ZScaleInterval())

<b>After successful testing, I incorporated the `CosmicRayLabel` class for labeling instead of using the commented code </b>

In [ ]:
# label, num_feat = lc.get_label(dq, bit_flag=8192, structure_element = np.ones((3,3)))
cr = CosmicRayLabel(fname)
cr.get_data()
cr.get_label()
# print('The number of cosmic ray labels: {}'.format(num_feat))

In [ ]:
num_cosmics = len(np.unique(cr.label.flatten())[1:])
print('The number of cosmic ray labels after thresholding: {}'.format(num_cosmics))


<hr style="height:3px;border:none;color:#333;background-color:#333;" />

<a id="total_energy"></a>
<h2>Total Energy Depoistion per Cosmic Ray </h2>
<p>Use the label as a mask on the SCI array, loop through each unique label and recored the sum of the corresponding pixel in the SCI array</p>

In [ ]:
def compute_total_cr_deposition_v1(label, sci):
    """Apply image label for a single chip to its corresponding science
    extension.
    
    The label should already have had the size threshold applied, with
    all CR smaller than threshold set to 0.
    
    Parameters
    ----------
    label -- image label with cr information
    sci -- science extension

    Returns
    -------
    data_per_pixel -- individual value of all pixels affected by CR
    data_per_cr -- total signal deposited by CR

    """

    unique_labels = np.unique(label)
    cr_sum = ndimage.sum(sci, labels=label, index=unique_labels[1:])
    cr_mean_energy = ndimage.mean(sci, labels=label, index=unique_labels[1:])
    return cr_sum, cr_mean_energy

In [ ]:
def compute_total_cr_deposition_v2(label, sci):
    """Apply image label for a single chip to its corresponding science
    extension.
    
    The label should already have had the size threshold applied, with
    all CR smaller than threshold set to 0.
    
    Parameters
    ----------
    label -- image label with cr information
    sci -- science extension

    Returns
    -------
    data_per_pixel -- individual value of all pixels affected by CR
    data_per_cr -- total signal deposited by CR

    """

    unique_labels = pd.Series(list(set(label.ravel())),name='cr_number')
    sorted_labels = unique_labels.sort_values(axis=0)
    # The first number will always be the background (i.e. 0)
    data = []
    for cr_num in sorted_labels[1:]:
        data.append(sci[label==cr_num])
    data_per_pixel = [a for datum in data for a in datum if a > 0]
    data_per_cr = [datum.sum() for datum in data]
    return data_per_pixel, data_per_cr

In [ ]:
%timeit compute_total_cr_deposition_v1(cr.label, sci)
cr_sum, cr_mean_energy = compute_total_cr_deposition_v1(cr.label, sci)

In [ ]:
start_time = time.time()
data_per_pixel, data_per_cr = compute_total_cr_deposition_v2(cr.label, sci)
end_time = time.time()
print('Total elapsed time: {:.3f} mins'.format((end_time - start_time)/60))

In [ ]:
fig, ax = plt.subplots(1,1)
data = ax.hist(np.log10(cr_sum),bins=50, range=(0,5))


In [ ]:
test = plt.hist(np.log10(data_per_cr),bins=50, range=(0,5))
plt.show()

<h3> Comparing the outputs </h3>
- let a = average energy deposited by a cosmic ray found `compute_total_cr_deposition_v1`
- let b = average energy deposited by a cosmic ray found `compute_total_cr_deposition_v2`

If a and b do not satisfy $|a - b| < 10^{-3}$, then next cell will raise an assertion error.


In [ ]:
print(np.mean(cr_sum), np.mean(data_per_cr))

In [ ]:
assert np.isclose(np.mean(cr_sum), np.mean(data_per_cr), rtol=0, atol=1e-3)


<h2>Results</h2>

From the above work we see the two methods are equivalent in their results, however the `ndimage` implementation has a significant boost in performance taking fractions of the time that my custom implementation works. So moving forward we will work with the `ndimage` implemenation.

<hr style="height:3px;border:none;color:#333;background-color:#333;" />


<a id="cr_shape_size"></a>
<h2>Size and Shape of Cosmic Ray</h2>

- Supply the label of the cosmic ray to the function `ndimage.find_objects()` to generate the smallest box containing a each individual cosmic ray identified in the label. 
    - returns a `tuple` of `slice` objects
    - First element is the `slice` along the row axis (y direction)
    - Second element is the `slice` along the column axis (x direction)
- Using the box, compute the moments of the cr distribution:
<ol>
    <li>$I_0 = \sum_{i} p_i$</li>
    <li>$I_x = \frac{1}{I_0} \sum_{i}p_i * x_i $</li>
    <li>$I_y = \frac{1}{I_0} \sum_{i} p_i * y_i $</li>
    <li>$I_{xx} = \frac{1}{I_0} \sum_{i}p_i(x_i - I_x)^2$</li>
    <li>$I_{yy} = \frac{1}{I_0} \sum_{i}p_i(y_i - I_y)^2$</li>
    <li>$I_{xy} = \frac{1}{I_0} \sum_{i}p_i(x_i - I_x)*(y_i - I_y)$</li>
</ol>
$$Size = \sqrt{\frac{I_{xx} + I_{yy}}{2}} $$
<p></p>
$$Anisotropy = \sqrt{\frac{(I_{xx} - I_{yy})^2 + 4I^2_{xy}}{(I_{xx} + I_{yy})^2}}$$

In [ ]:
def compute_shape(label, sci):
    cr_locs = ndimage.find_objects(label)
    return cr_locs

In [ ]:
sizes = np.bincount(cr.label.ravel())
index = np.argmax(sizes[1:])
print(index)


In [ ]:
cr_locs = compute_shape(cr.label, sci)

### Custom Center of Intensity Calculation

In [ ]:
edge_cr_index = 1977
cr_locs[index]

In [ ]:
tmp = cr_locs[index]
y_slice = tmp[0]
x_slice = tmp[1]
print(y_slice.start, y_slice.stop)
print(x_slice.start, x_slice.stop)

In [ ]:
y_coords = np.linspace(y_slice.start, y_slice.stop, int(y_slice.stop - y_slice.start)+1)

In [ ]:
y_coords

In [ ]:
x_coords = np.linspace(x_slice.start, x_slice.stop, int(x_slice.stop - x_slice.start)+1)

In [ ]:
x_coords

In [ ]:
xx, yy = np.meshgrid(x_coords, y_coords)

In [ ]:
print(xx.shape, xx.shape[0]*xx.shape[1])

In [ ]:
xx

In [ ]:
yy

In [ ]:
# Create a list of all of the possible coordinate locations on the grid
positions = np.vstack([yy.ravel(), xx.ravel()])

In [ ]:
grid_coords = list(zip(map(int, positions[0]), map(int, positions[1])))
# for pos in grid_coords:
#     print(pos[0], pos[1])

In [ ]:
I_0 = ndimage.sum(sci, labels=cr.label, index=[index+1])[0]
print(I_0)

In [ ]:
I_x = 0
I_y = 0
cr_coords = []
for r in grid_coords:
    if cr.label[r[0]][r[1]] == index+1:
#         print(sci[r[0]][r[1]])
        I_x += sci[r[0]][r[1]] * r[1]
        I_y += sci[r[0]][r[1]] * r[0]
        cr_coords.append(r)

In [ ]:
custom_cm = (I_y/I_0, I_x/I_0)

In [ ]:
shifted_origin = (I_y/I_0 - y_slice.start, I_x/I_0 - x_slice.start)

In [ ]:
print(custom_cm)
print(shifted_origin)

***

In [ ]:
extrema = ndimage.extrema(sci, labels=cr.label, index=[index+1])

In [ ]:
extrema

In [ ]:
min_pos = extrema[-2][0]
max_pos = extrema[-1][0]

In [ ]:
min_pos

In [ ]:
min_pos1 = (min_pos[0] - y_slice.start,min_pos[1] - x_slice.start)
max_pos1 = (max_pos[0] - y_slice.start, max_pos[1] - x_slice.start)

In [ ]:
min_pos1

In [ ]:
box_sci_slice = sci[cr_locs[index][0], cr_locs[index][1]]
box_label_slice = cr.label[cr_locs[index][0], cr_locs[index][1]]

In [ ]:
print(box_sci_slice[0][0], box_sci_slice[-1][0],box_sci_slice[0][-1],box_sci_slice[-1][-1])

In [ ]:
def mk_patch(r_cm, c='red'):
    CR_center = patches.Rectangle((r_cm[1]-0.5,r_cm[0]-0.5), 
                              width=1, height=1, 
                              alpha=1.0, fill=False,
                              linewidth=1.75, color=c)
    return CR_center

In [ ]:
fig, (ax, ax1) = plt.subplots(1,2, sharex=True, sharey=True)
ax.imshow(box_sci_slice, norm=norm, cmap='gray', origin='lower')
ax1.imshow(box_label_slice, cmap=random_cmap(len(cr_locs)),origin='lower')
patch1 = mk_patch(r_cm= shifted_origin ,c='red')
patch2 = mk_patch(r_cm = max_pos1, c='green')
ax.add_patch(patch1)
ax.add_patch(patch2)

In [ ]:
r_cm = ndimage.measurements.center_of_mass(sci, labels=cr.label, index=np.unique(cr.label)[1:])
# r_cm = ndimage.measurements.center_of_mass(sci, labels=label, index=[index+1])
print(len(r_cm))

In [ ]:
# (row variable, column variable) <--> (y, x)
# r_cm.append(custom_cm)
# r_cm = list(r_cm[0])
print(r_cm)

In [ ]:
print(cr_locs[index+1])

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(1,1,1)
ax.imshow(sci, norm=norm, cmap='gray',origin='lower')

for cm in r_cm[:1000]:
    patch = mk_patch(cm)
    ax.add_patch(patch)

# Use CM to compute $I_{xx}$ and $I_{yy}$, then compute the size

In [ ]:
def mk_grid(slice_tuple):
    """Build a meshgrid from a tuple of python slice objects

    Parameters
    ----------
    slice_tuple

    Returns
    -------

    """
    y_slice = slice_tuple[0]
    x_slice = slice_tuple[1]


    if int(x_slice.stop) == 4096:
        print('Cosmic ray struck the edge!!!!!!!!!!!!')
        y_coords = np.linspace(y_slice.start, y_slice.stop,
                               int(y_slice.stop - y_slice.start)+1)

        x_coords = np.linspace(x_slice.start, x_slice.stop,
                               int(x_slice.stop - x_slice.start)+1,
                               endpoint=False)

    elif int(y_slice.stop)  == 4096:
        print('Cosmic ray struck the edge!!!!!!!!!!!!')
        y_coords = np.linspace(y_slice.start, y_slice.stop,
                               int(y_slice.stop - y_slice.start)+1,
                               endpoint=False)
        x_coords = np.linspace(x_slice.start, x_slice.stop,
                               int(x_slice.stop - x_slice.start)+1)

    else:
        y_coords = np.linspace(y_slice.start, y_slice.stop,
                               int(y_slice.stop - y_slice.start) + 1)

        x_coords = np.linspace(x_slice.start, x_slice.stop,
                               int(x_slice.stop - x_slice.start) + 1)

    xx, yy = np.meshgrid(x_coords, y_coords)
    positions = np.vstack([yy.ravel(), xx.ravel()])
    grid_coords = list(zip(map(int, positions[0]), map(int, positions[1])))

    return grid_coords

In [ ]:
def compute_first_moment(label, sci):
    """

    Parameters
    ----------
    sci
    label

    Returns
    -------

    """
    r_cm = ndimage.measurements.center_of_mass(sci,
                                               labels=label,
                                               index=np.unique(label)[1:])
    print(len(r_cm))
    return r_cm


def compute_second_moment(I_0, sci, label, I_ci, grid_coords, index):
    """

    Parameters
    ----------
    I_0
    p_i
    r_i
    I_ci

    Returns
    -------

    """
    second_moment = [0,0]
    for r_i in grid_coords:
        if label[r_i[0]][r_i[1]] == index:
            second_moment += (1/I_0) * sci[r_i[0]][r_i[1]] * (np.asarray(r_i) - np.asarray(I_ci))**2

    return np.asarray(second_moment)


In [ ]:
grid_coords = mk_grid(cr_locs[index])

In [ ]:
grid_coords

In [ ]:
I_0 = ndimage.sum(sci, labels=cr.label, index=[index+1])[0]
print(I_0)
second_moment = compute_second_moment(I_0, sci, cr.label, r_cm[index+1],
                                      grid_coords, index+1)

In [ ]:
np.sqrt(second_moment.sum()/2)

In [ ]:
I_0 = ndimage.sum(sci, labels=cr.label, index=[index+1])[0]
I_yy = 0
I_xx = 0
for r in grid_coords:
    if label[r[0]][r[1]] == index+1:
#         print(sci[r[0]][r[1]])
        I_xx += compute_I_xx(I_0, sci[r[0]][[r[1]]], r[1], r_cm[0][1])
        I_yy += compute_I_yy(I_0, sci[r[0]][[r[1]]], r[0], r_cm[0][0])
        

### Computing the $I_{xy}$
$I_{xy} = \frac{1}{I_0} \sum_{i}p_i(x_i - I_x)*(y_i - I_y)$

In [ ]:
def compute_cross_moment(I_0, sci, label, I_ci, grid_coords, index):
    I_xy = 0
    for r_i in grid_coords:
        if label[r_i[0]][r_i[1]] == index:
            I_xy += (1/I_0) * sci[r_i[0]][r_i[1]] * (r_i[0] - I_ci[0]) * (r_i[1] - I_ci[1])
    return I_xy

### Testing computation of sizes

In [ ]:
I_0 = ndimage.sum(sci, labels=label, index=[index])[0]

In [ ]:
grid_coords = cs.mk_grid(cr_locs[index-1])

In [ ]:
grid_coords

In [ ]:
cross_momements = compute_cross_moment(I_0, sci, label, r_cm[0],grid_coords, index+1)

In [ ]:
R_cm = cs.compute_first_moment(label, sci)

In [ ]:
len(R_cm)

In [ ]:
cr_locs = ndimage.find_objects(label)
cr_locs = [cr for cr in cr_locs if cr is not None]

In [ ]:
cr_deposition = cs.compute_total_cr_deposition_v1(label, sci)

In [ ]:
len(R_cm), len(cr_deposition), len(cr_locs), len(np.unique(label)[1:])

In [ ]:
sizes = {}
second_moments = {}
loop_obj = zip(np.unique(label)[1:], R_cm, cr_deposition, cr_locs)
for int_id, r_cm, I_0, loc in loop_obj:
    grid_coords = cs.mk_grid(loc)
    second_moment = cs.compute_second_moment(I_0, sci, label,
                                              r_cm, grid_coords, int_id)
    second_moments[int_id] = second_moment
    sizes[int_id] = np.sqrt(second_moment.sum()/2)

In [ ]:
cosmic_ray_sizes = list(sizes.values())

In [ ]:
np.max(cosmic_ray_sizes)

In [ ]:
cross_moments = {}
loop_obj = zip(np.unique(label)[1:], R_cm, cr_deposition, cr_locs)
for int_id, r_cm, I_0, loc in loop_obj:
    grid_coords = cs.mk_grid(loc)
    cross_moment = compute_cross_moment(I_0, sci, label, r_cm, grid_coords, int_id)
    cross_moments[int_id] = cross_moment
    

In [ ]:
print(cross_moments[index],second_moments[index])
index

$$Anisotropy = \sqrt{\frac{(I_{xx} - I_{yy})^2 + 4I^2_{xy}}{(I_{xx} + I_{yy})^2}}$$


In [ ]:
def compute_anistropy(second_moments, cross_moment):
    return np.sqrt(((second_moments[0] - second_moments[1])**2 + 4*cross_moment**2)/(second_moment.sum())**2)

In [ ]:
compute_anistropy(second_moments[index+1], cross_moments[index+1])

<h2>Results</h2>

From the above test we see that the centroiding calculation works great, and all of the higher order moment can be readilty calculated building on the zeroth, first, and second moments. All of the code written here has been implemented in a class `ComputeStats()`

<hr style="height:3px;border:none;color:#333;background-color:#333;" />

<a id="normal_incidence"></a>
<h2>Angle of incidence wrt to plane of CCD</h2>
<p> The idea here is to use a thickness map of the CCD to compute the angle of incidence of the cosmic ray. By counting up the number of pixels the cosmic ray has traversed, we can use the average thickness ($\Delta z$) of the detector and the average pixel size ($\overline{s}$) to solve for theta:</p>
$$ tan(\theta)=\frac{\Delta z}{n_{pix}*\overline{s}}$$



<a id="polar_angle"></a>
<h2> Polar angle wrt to defined CCD axis</h2>
<p>The idea here is to use the results returned by `find_objects` to compute the angle between the CCD axes